# Using Simplicit's Easy API To Simulate Avocado Mesh
A simple way to use the simplicit's code base. We can create a simple object, train it, simulate it and visualize all in a very few lines of code.

In [7]:
import copy, math, os, sys, logging, threading
from typing import List, Tuple
from pathlib import Path

import numpy as np
import torch
import kaolin as kal

from IPython.display import display
from ipywidgets import Button, HBox, VBox

logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)
logger = logging.getLogger(__name__)

sys.path.append(str(Path("..")))
from tutorial_common import COMMON_DATA_DIR

def sample_mesh_path(fname):
    return os.path.join(COMMON_DATA_DIR, 'meshes', fname)

def print_tensor(t, name='', **kwargs):
    print(kal.utils.testing.tensor_info(t, name=name, **kwargs))

## Loading Geometry
Simplicies works with any geometry. Meshes, pointclouds, SDFs, Gaussian splats, and more. For this example we 

In [8]:
# Import and triangulate to enable rasterization; move to GPU
mesh = kal.io.import_mesh(sample_mesh_path('avocado.obj'), triangulate=True).cuda()
# Normalize so it is easy to set up default camera
mesh.vertices = kal.ops.pointcloud.center_points(mesh.vertices.unsqueeze(0), normalize=True).squeeze(0) 
orig_vertices = mesh.vertices.clone()  # Also save original undeformed vertices

# Inspect
print(mesh)

SurfaceMesh object with batching strategy NONE
            vertices: [406, 3] (torch.float32)[cuda:0]  
               faces: [682, 3] (torch.int64)[cuda:0]  
             normals: [377, 3] (torch.float32)[cuda:0]  
    face_normals_idx: [682, 3] (torch.int64)[cuda:0]  
                 uvs: [406, 2] (torch.float32)[cuda:0]  
        face_uvs_idx: [682, 3] (torch.int64)[cuda:0]  
material_assignments: [682] (torch.int16)[cuda:0]  
           materials: list of length 1
       face_vertices: if possible, computed on access from: (faces, vertices)
        face_normals: if possible, computed on access from: (normals, face_normals_idx) or (vertex_normals, faces) or (vertices, faces)
            face_uvs: if possible, computed on access from: (uvs, face_uvs_idx)
      vertex_normals: if possible, computed on access from: (faces, face_normals)
     vertex_tangents: if possible, computed on access from: (faces, face_vertices, face_uvs, vertex_normals)
       vertex_colors: if possible, comput

## Setup
Lets set up the vertices and material parameters of our object

In [4]:
# Physics material parameters
soft_youngs_modulus = 1e5
poisson_ratio = 0.45
rho = 500  # kg/m^3
approx_volume = 0.5  # m^3

# Points sampled over the object
pts = mesh.vertices
yms = torch.full((pts.shape[0],), soft_youngs_modulus, device="cuda")
prs = torch.full((pts.shape[0],), poisson_ratio, device="cuda")
rhos = torch.full((pts.shape[0],), rho, device="cuda")


## Create a SimplicitsObject
We make a "simplicits object" using our api. Then we train it for num_steps.

In [5]:
# Initialize and train a Simpicits object to enable simulation
sim_obj = kal.physics.simplicits.SimplicitsObject(pts, yms, prs, rhos, torch.tensor([approx_volume], dtype=torch.float32, device="cuda"), num_handles=5)
sim_obj.train(num_steps=10000)

DEBUG:kaolin.physics.simplicits.easy_api:Training step: 0, le: 428.7832946777344, lo: 32387982.0
DEBUG:kaolin.physics.simplicits.easy_api:Training step: 100, le: 23.801427841186523, lo: 154803.328125
DEBUG:kaolin.physics.simplicits.easy_api:Training step: 200, le: 112.9085464477539, lo: 116969.1796875
DEBUG:kaolin.physics.simplicits.easy_api:Training step: 300, le: 264.17999267578125, lo: 80777.2734375
DEBUG:kaolin.physics.simplicits.easy_api:Training step: 400, le: 308.72802734375, lo: 80147.8515625
DEBUG:kaolin.physics.simplicits.easy_api:Training step: 500, le: 221.305908203125, lo: 80178.9375
DEBUG:kaolin.physics.simplicits.easy_api:Training step: 600, le: 271.9401550292969, lo: 79970.1328125
DEBUG:kaolin.physics.simplicits.easy_api:Training step: 700, le: 165.41661071777344, lo: 77258.0703125
DEBUG:kaolin.physics.simplicits.easy_api:Training step: 800, le: 857.9734497070312, lo: 40415.76953125
DEBUG:kaolin.physics.simplicits.easy_api:Training step: 900, le: 632.1121215820312, lo: 

## Create a Scene
Now that we're trained the object. Lets simulate it by creating a scene

In [ ]:
# Create a default scene
scene = kal.physics.simplicits.SimplicitsScene() # default empty scene
scene.max_newton_steps = 3 #Convergence might not be guaranteed, but runs very fast

In [ ]:
# Add our object to the scene. The scene copies it into an internal SimulatableObject utility class
obj_idx = scene.add_object(sim_obj)

In [ ]:
# Add gravity to the scene
scene.set_scene_gravity(acc_gravity=torch.tensor([0, 9.8, 0]))
# Add floor to the scene
scene.set_scene_floor(floor_height=-0.8, floor_axis=1, floor_penalty=1000)

In [ ]:
# Make the object softer by updating the material parameter
scene.set_object_materials(0, yms=torch.tensor(5e4, device='cuda', dtype=torch.float))

## Thats it! Onto display
All we need to do now is display the object using Kaolin's in-notebook visualizer.

In [ ]:
resolution = 512
camera = kal.render.easy_render.default_camera(resolution).cuda()

azimuth = torch.zeros((1,), device='cuda')
elevation = torch.full((1,), math.pi / 3., device='cuda')
amplitude = torch.full((1, 3), 3., device='cuda')
sharpness = torch.full((1,), 5., device='cuda')

def current_lighting():
    direction = kal.render.lighting.sg_direction_from_azimuth_elevation(azimuth, elevation)
    return kal.render.lighting.SgLightingParameters(
        amplitude=amplitude, sharpness=sharpness, direction=direction)
    
def render(in_cam, **kwargs):
    active_pass=kal.render.easy_render.RenderPass.render
    render_res = kal.render.easy_render.render_mesh(in_cam, mesh, lighting=current_lighting(), **kwargs)
    
    img = render_res[active_pass]
    background_mask = (render_res[kal.render.easy_render.RenderPass.face_idx] < 0).bool()
    img2 = torch.clamp(img, 0, 1)[0]
    img2[background_mask[0]] = 1
    final = (img2 * 255.).to(torch.uint8)
    return {"img":final, "face_idx": render_res[kal.render.easy_render.RenderPass.face_idx].squeeze(0).unsqueeze(-1)}

def fast_render(in_cam, factor=8):
    lowres_cam = copy.deepcopy(in_cam)
    lowres_cam.width = in_cam.width // factor
    lowres_cam.height = in_cam.height // factor
    return render(lowres_cam)

def run_sim():
    scene.reset_object(0)

    for s in range(100):
        with visualizer.out:
            scene.run_sim_step()
            print(".", end="")
        mesh.vertices = scene.get_object_deformed_pts(obj_idx).squeeze()
        visualizer.render_update()

def start_simulation(b):
    # run_sim()
    t = threading.Thread(target=run_sim, daemon=True)
    t.start()

scene.reset_object(0)
button = Button(description='Run Sim')
button.on_click(start_simulation)
visualizer = kal.visualize.IpyTurntableVisualizer(
    resolution, resolution, copy.deepcopy(camera), render, fast_render=fast_render,
    max_fps=24, world_up_axis=1)
display(HBox([visualizer.canvas, button]), visualizer.out)